# First run the geolife.py script to generate trajectories

In [2]:
#pragma cling add_include_path("/usr/src/movetk/tutorials")

In [3]:
#pragma cling add_include_path("/usr/local/include")

In [4]:
#include "movetk/utils/Geolife.h"

In file included from input_line_15:1:
In file included from /usr/local/include/movetk/utils/Geolife.h:21:
In file included from /usr/local/include/movetk/io/ProbeReader.h:40:
In file included from /miniconda/include/boost/iostreams/copy.hpp:26:
/miniconda/include/boost/bind.hpp:36:1: warning: The practice of declaring the Bind placeholders (_1, _2, ...) in the global namespace is deprecated. Please use
      &lt;boost/bind/bind.hpp&gt; + using namespace boost::placeholders, or define BOOST_BIND_GLOBAL_PLACEHOLDERS to retain the current
      behavior. [-W#pragma-messages]
BOOST_PRAGMA_MESSAGE(
^
/miniconda/include/boost/config/pragma_message.hpp:24:34: note: expanded from macro &#39;BOOST_PRAGMA_MESSAGE&#39;
# define BOOST_PRAGMA_MESSAGE(x) _Pragma(BOOST_STRINGIZE(message(x)))
                                 ^
&lt;scratch space&gt;:7:2: note: expanded from here
 message(&quot;The practice of declaring the Bind placeholders (_1, _2, ...) &quot; &quot;in the global namespace is depreca

In [5]:
?ProbeReader

http://127.0.0.1/movetk.reference/class_probe_reader.html

In [6]:
// Create trajectory reader
std::unique_ptr<movetk::io::ProbeReader<ProbeTraits>> probe_reader;

In [5]:
std::string fName = "/usr/src/movetk/tutorials/out/117.csv";

In [6]:
probe_reader = movetk::io::ProbeReaderFactory::create<ProbeTraits>(fName.c_str());

In [7]:
?TrajectoryReader

http://192.168.99.100/movetk.reference/class_trajectory_reader.html

In [8]:
//type of probe iterator
using ProbeInputIterator = decltype(probe_reader->begin());

In [9]:
auto trajectory_reader = movetk::io::TrajectoryReader<TrajectoryTraits, ProbeInputIterator>(probe_reader->begin(), probe_reader->end());

In [11]:
std::cout<<"{\"type\":\"FeatureCollection\",\"features\":[";

for (auto tit = trajectory_reader.begin(); tit!= trajectory_reader.end();){
    auto trajectory = *tit;
    if (trajectory.size() == 1)
    {
            ++tit;
            continue;
    }
   //write GeoJSON
   GeoJSONGeometry geom;
   GeoJSONProperties prop;
   GeoJSONFeature feat;
   auto lats = trajectory.template get<ProbeTraits::ProbeColumns::LAT>();
   auto lons = trajectory.template get<ProbeTraits::ProbeColumns::LON>();
   rapidjson::Document geometry = geom(std::begin(lats), std::end(lats),
                                        std::begin(lons));
    
   rapidjson::Document properties = prop();
    
   rapidjson::Document feature = feat(geometry, properties);
   rapidjson::StringBuffer strbuf;
   rapidjson::Writer<rapidjson::StringBuffer> writer(strbuf);
   feature.Accept(writer);
   std::cout << strbuf.GetString();
   strbuf.Clear();  
   
    if (++tit == trajectory_reader.end())
            continue;
    else
            std::cout << ",";
}

std::cout << "]}" << std::endl;